# Extracting and Transforming Metadata

This notebook illustrates a second step in the "transform" process.
Although there are many possible data transformation pathways, this demonstration
illustrates a process that converts metadata into an open, non-proprietary,
text-based format: the CSV file.

As throughout, the general process here follows the generalized "Extract - Transform - Load" process, which is frequently the abstract model for pulling data from one system, transporting, cleaning, and outputting to another system, which remains the overall workflow.

## Learning objectives

After completing the assignment associated with this notebook, you should: 

* Use programming (Python) to work with data supplied by an API in JSON format to manage and transform useful parts of that data into a CSV format.
* Create ingest-ready collection metadata that conforms to Dublin Core and other digital collection metadata standards, which can be used to load content into another site (in this case, a new collection platform, like Omeka S or CollectionBuilder).

## Introduction

The main steps outlined in this notebook are as follows:

* **Transform the metadata.** This step assumes you have already developed a plan for transforming your data, which is your Metadata Application Profile, or MAP.
  1. Develop your transformation script with a small subset of the metadata. In this case, one record.
  1. Transform the data you've gathered in JSON into a CSV file according to the metadata crosswalk you've developed. The goal in this step is to create a CSV that we can use to import items into your Omeka site (using the CSV Import module). Note that the code outlined here suggests how all of these data elements may be extracted and transformed, but it does not necessarily output all of the elements that you will need to complete your assignment. In other words, there is still work to do to complete this code, but you are welcome to adopt or reuse the code here.

# Get collection list

In [1]:
import csv
import json

# for working with local files
import glob
import os
from os.path import join

# Transformation Part 1: Testing

At a high level, the transformation step involves the creation of code or another implementation workflow, which will search the item metadata files downloaded previously, extract the target fields identified in the MAP, then write that information to a CSV for ingest or loading into a new presentation platform.

First, develop a search pattern for identifying the desired JSON files. Here, you create a list of the files that you want to transform, called `list_of_item_metadata_files`. 

**Reminder:** This step builds on your terminal skills! (And builds on your understanding of regular expressions and shell navigation. Note, however, these are technically file path expansions, not actual regular expressions. The general idea of creating a pattern and asking the computer to respond with a list of results that meet your criteria, is similar.)

In [2]:
current_loc = os.getcwd()

print(current_loc)

c:\Users\kenda\Desktop\si676-networked-information-services\Collection Project


In [3]:
metadata_file_path = os.path.join('item-metadata-gardens')

print(metadata_file_path)

item-metadata-gardens


The next cell uses the `glob` library, which supports the use of file path expanders
to look for patterns in file paths. In this case, the previous item metadata exraction
wrote files that had the pattern `item_metadata-[item-identifier].json`. 
So, to match any pattern for the `item-identifier` section, `glob` allows
the use of the `*` (asterisk) character to match any pattern:

In [4]:
file_count = 0

for file in glob.glob('item-metadata-gardens/item_metadata-*.json'):
    file_count += 1
    print(file)

print('found',file_count)

item-metadata-gardens\item_metadata-afc1999008.afc1999008_crf_mhc05407.json
item-metadata-gardens\item_metadata-cph.3a25266.json
item-metadata-gardens\item_metadata-cph.3c30923.json
item-metadata-gardens\item_metadata-cph.3f05737.json
item-metadata-gardens\item_metadata-cph.3g05158.json
item-metadata-gardens\item_metadata-ds.03659.json
item-metadata-gardens\item_metadata-ds.13761.json
item-metadata-gardens\item_metadata-fsa.8a19113.json
item-metadata-gardens\item_metadata-fsa.8b14071.json
item-metadata-gardens\item_metadata-fsa.8b14514.json
item-metadata-gardens\item_metadata-fsa.8c19781.json
item-metadata-gardens\item_metadata-fsa.8d21091.json
item-metadata-gardens\item_metadata-fsa.8d31670.json
item-metadata-gardens\item_metadata-ggbain.01389.json
item-metadata-gardens\item_metadata-hhh.ca1376.photos.json
item-metadata-gardens\item_metadata-hhh.ma1202.sheet.json
item-metadata-gardens\item_metadata-highsm.13145.json
item-metadata-gardens\item_metadata-highsm.16140.json
item-metadata-g

In [5]:
list_of_item_metadata_files = list()
for file in glob.glob('item-metadata-gardens/item_metadata-*.json'):
    list_of_item_metadata_files.append(file)

In [6]:
len(list_of_item_metadata_files)

48

In [7]:
# quick duplicate check
list_of_item_metadata_files.sort()

for file in list_of_item_metadata_files:
    print(file)

item-metadata-gardens\item_metadata-afc1999008.afc1999008_crf_mhc05407.json
item-metadata-gardens\item_metadata-cph.3a25266.json
item-metadata-gardens\item_metadata-cph.3c30923.json
item-metadata-gardens\item_metadata-cph.3f05737.json
item-metadata-gardens\item_metadata-cph.3g05158.json
item-metadata-gardens\item_metadata-ds.03659.json
item-metadata-gardens\item_metadata-ds.13761.json
item-metadata-gardens\item_metadata-fsa.8a19113.json
item-metadata-gardens\item_metadata-fsa.8b14071.json
item-metadata-gardens\item_metadata-fsa.8b14514.json
item-metadata-gardens\item_metadata-fsa.8c19781.json
item-metadata-gardens\item_metadata-fsa.8d21091.json
item-metadata-gardens\item_metadata-fsa.8d31670.json
item-metadata-gardens\item_metadata-ggbain.01389.json
item-metadata-gardens\item_metadata-hhh.ca1376.photos.json
item-metadata-gardens\item_metadata-hhh.ma1202.sheet.json
item-metadata-gardens\item_metadata-highsm.13145.json
item-metadata-gardens\item_metadata-highsm.16140.json
item-metadata-g

To develop your data transformation and metadata profile, 
first you need to explore the information that you have about each item. 
To do this, explore one item to understand how the information is structured.
How do you open the json? How is it structured? Where is the information you want?

In [8]:
# try first with one file, can you open the json, can you see what elements are in the json?
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as item:
    # what are we looking at?
    print('file:',list_of_item_metadata_files[0],'\n')
    
    # load the item data
    item_data = json.load(item)
    
    for element in item_data.keys():
        print(element,':',item_data[element])

file: item-metadata-gardens\item_metadata-afc1999008.afc1999008_crf_mhc05407.json 

_version_ : 1728618448837148672
access_restricted : False
aka : ['http://www.loc.gov/item/cmns001121/', 'https://hdl.loc.gov/loc.afc/afccmns.mhc05407', 'http://www.loc.gov/resource/afc1999008.afc1999008_crf_mhc05407/', 'http://www.loc.gov/item/afc1999008.afc1999008_crf_mhc05407/']
call_number : ['AFC 1999/008: CRF-MH-C054-07']
campaigns : []
contributor_names : ['Jarrell, Bob (Depicted)', 'Hufford, Mary, 1952- (Photographer)']
contributors : [{'hufford, mary': 'https://www.loc.gov/search/?fa=contributor:hufford,+mary&fo=json'}, {'jarrell, bob': 'https://www.loc.gov/search/?fa=contributor:jarrell,+bob&fo=json'}]
created_published : ['June 28, 1996']
date : 1996-06-28
dates : [{'1996-06-28': 'https://www.loc.gov/search/?dates=1996-06-28/1996-06-28&fo=json'}]
digital_id : ['http://hdl.loc.gov/loc.afc/afccmns.mhc05407']
digitized : True
display_offsite : True
extract_timestamp : 2021-02-23T13:11:31.639Z
ext

Look around in the dictionary a bit more:

In [9]:
item_data.keys()

dict_keys(['_version_', 'access_restricted', 'aka', 'call_number', 'campaigns', 'contributor_names', 'contributors', 'created_published', 'date', 'dates', 'digital_id', 'digitized', 'display_offsite', 'extract_timestamp', 'extract_urls', 'format', 'genre', 'group', 'hassegments', 'id', 'image_url', 'index', 'item', 'language', 'languages', 'location', 'locations', 'medium', 'mime_type', 'notes', 'online_format', 'original_format', 'other_formats', 'partof', 'repository', 'resources', 'rights', 'score', 'shard', 'shelf_id', 'source_collection', 'subject', 'subject_headings', 'subjects', 'timestamp', 'title', 'type', 'url'])

For the development of your metadata transformation, you're looking for 
how to extract the elements identified in the MAP table. For example, which date fields do you want and where are they? Where will you find the format information?

In [10]:
# can you get the date?
print('\ndate:',item_data['date'], type(item_data['date']))
# can you get the format?
print('\nformat:',item_data['format'][0], type(item_data['format']))


date: 1996-06-28 <class 'str'>

format: {'photo, print, drawing': 'https://www.loc.gov/search/?fa=original_format:photo,+print,+drawing&fo=json'} <class 'list'>


## Test: Try it with one example

First, try to set up the extract process with one example. This may get more complicated later since you don't know yet if every item has the same metadata attributes in the JSON. But start with some basics and build up from there. 

For a first pass, look out for these items, and find where in the JSON you can locate them:

* 'item_id'
* 'title'
* 'date' 
* 'source_url'
* 'phys_format'
* 'dig_format'
* 'rights'

_Hint: use the JSON viewer in JupyterLab, use an extension in VSCode, or use a browser to look through sample JSON. The block below uses item `cph.3b41963`._

You may need to use try/except patterns to create workarounds for cases where some items may not have exactly the same attributes that you've identified in your test cases.

In [11]:
# set up the containers to create the csv of all the item fields
# file for csv to read out
collection_info_csv = 'collection_items_data_gardens.csv'

# set up a list for the columns in your csv; 
# your goal should be to automate this, but . . . 
# it works for demonstration as you set up the crosswalk
headers = ['source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'dig_format', 'rights']

# try first with one file
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as data:
    # load the item data
    item_data = json.load(data)
    
    # extract the data you want
    # for checking purposes, add in the source of the info
    source_file = str(list_of_item_metadata_files[0])
    # make sure there's some unique and stable identifier
    try:
        item_id = item_data['library_of_congress_control_number']
    except:
        item_id = item_data['url'].split('/')[-2]
        
    title = item_data['title']
    date = item_data['date']
    source_url = item_data['url']
    
    try:
        phys_format = item_data['format'][0]
    except:
        phys_format = 'Not found'
        
    try:
        dig_format = item_data['online_format'][0]
    except:
        dig_format = 'Not found'
        
    mime_type = item_data['mime_type']
    
    try:
        rights = item_data['rights_information']
    except:
        rights = 'Undetermined'


    # dictionary for the rows
    row_dict = dict()
    
    # look for the item metadata, assign it to the dictionary; 
    # start with some basic elements likely (already enumerated in the headers list) :
    # source file
    row_dict['source_file'] = source_file
    # identifier
    row_dict['item_id'] = item_id
    # title
    row_dict['title'] = title
    # date
    row_dict['date'] = date
    # link
    row_dict['source_url'] = source_url
    # format
    row_dict['phys_format'] = phys_format
    # digital format
    row_dict['dig_format'] = dig_format
    #rights
    row_dict['rights'] = rights 
    print('created row dictionary:',row_dict)

    # write to the csv
    with open(collection_info_csv, 'w', encoding='utf-8') as fout:
        writer = csv.DictWriter(fout, fieldnames=headers, lineterminator='\n')
        writer.writeheader()
        writer.writerow(row_dict)
        print('wrote',collection_info_csv)

created row dictionary: {'source_file': 'item-metadata-gardens\\item_metadata-afc1999008.afc1999008_crf_mhc05407.json', 'item_id': 'cmns001121', 'title': "Bob Jarrell's vegetable garden", 'date': '1996-06-28', 'source_url': 'https://www.loc.gov/item/cmns001121/', 'phys_format': {'photo, print, drawing': 'https://www.loc.gov/search/?fa=original_format:photo,+print,+drawing&fo=json'}, 'dig_format': 'image', 'rights': 'Undetermined'}
wrote collection_items_data_gardens.csv


You're now developing the structure of the CSV file that will import items into your Omeka S site. The CSV import module supports the loading of item files via a URL. This provides the location of a file (in this case, an image), which Omeka will copy into its database and attach to your item. This means that it isn't necessary to upload individual files after or during metadata creation. 

To allow this, you need to find a direct url to a good image file for the item. There are multiple options, and the code below demonstrates looking for the url to a medium-sized image of an item:

In [12]:
collection_info_csv = 'collection_items_data_gardens.csv'

# set up a list for the columns in your csv; in future, this should be more automated but this works for now as you set up the crosswalk
headers = ['source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'dig_format', 'rights']

# try first with one file
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as data:
    # load the item data
    item_data = json.load(data)
    
    print(item_data['image_url'][3])

https://tile.loc.gov/image-services/iiif/service:afc:afc1999008:afc1999008_crf_mhc05407/full/pct:50/0/default.jpg#h=1388&w=2048


# Transformation Part 2: Write your CSV

The goal of this final step is to create a CSV file, which will be possible to import into your Omeka site. It may seem like it's taken a long time to get to this point... but remember, when this works you will be importing around 60 items into the site at one time, so if you can get all of this to work for an even larger set of materials, you will be saving quite a lot of time in the future when you need to import items. Even if you were to collect the items piecemeal, which would need a different workflow than illustrated here, you can accomplish similar goals by recording metadata for each item consistently and in a spreadsheet, which you can then use to import the items in batch.

So now that your transformation script is tested, the goal is to extend this to the whole set by looping through each of the desired JSON files:

In [13]:
# for purposes of demonstration, use this block to make sure there isn't already a list file:

items_data_file = os.path.join('collection_items_data_gardens.csv')

if os.path.isfile(items_data_file):
    os.unlink(items_data_file)
    print('removed',items_data_file)

# clear row_dict
row_dict = {}

# get date string for upload date
from datetime import date
date_string_for_today = date.today().strftime('%Y-%m-%d') # see https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior

# set up the containers to create the csv & counters
# file for csv to read out
collection_info_csv = os.path.join('collection_items_data_gardens.csv')
file_count = 0
items_written = 0
error_count = 0

# add in a couple of extras for Omeka, including item type and date uploaded

# set up a list for the columns in your csv; in future, this should be more automated but this works for now as you set up the crosswalk
headers = ['item_type', 'date_uploaded', 'source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'dig_format', 'rights', 'image_url',
           'creators', 'description', 'subject_headings', 'source_collection', 'url', 'notes']

# now, adapt the previous loop to open each file:
for file in list_of_item_metadata_files:
    file_count += 1
    print('opening',file)
    with open(file, 'r', encoding='utf-8') as item:
        # load the item data
        try:
            item_data = json.load(item)
        except:
            print('error loading',file)
            error_count += 1
            continue

        # extract/name the data you want
        # item type
        item_type = 'Item'
        # date uplaoded
        date_uploaded = date_string_for_today
        # for checking purposes, add in the source of the info
        source_file = str(file)
        # make sure there's some unique and stable identifier
        try:
            item_id = item_data['library_of_congress_control_number']
        except:
            item_id = item_data['url'].split('/')[-2]
            
        title = item_data['title']
        
        try:
            date = item_data['date']
        except:
            date = 'Not found'
            
        source_url = item_data['url']
        
        try:
            phys_format = item_data['format'][0]
        except:
            phys_format = 'Not found'
            
        try:
            dig_format = item_data['online_format'][0]
        except:
            dig_format = 'Not found'
            
        mime_type = item_data['mime_type']
        
        try:
            rights = item_data['rights_information']
        except:
            rights = 'Undetermined'
        
        try:
            image_url = item_data['image_url'][-1]
        except:
            image_url = 'Did not identify a URL.'
        
        try:
            creators = item_data['item']['contributors']
        except:
            try:
                creators = []
                for contributor in item_data['contributors']:
                    creators.extend(contributor.keys())
            except:
                creators = []

        try:
            description = item_data['summary']
        except:
            description = 'Not found'
        
        try:
            subject_headings = item_data['subject_headings']
        except:
            subject_headings = []
        
        try:
            source_collection = item_data['source_collection']
        except:
            source_collection = 'Not found'
        
        try:
            url = item_data['link']
        except:
            url = 'Not found'
        
        try:
            notes = item_data['notes']
        except:
            notes = []
        
        # dictionary for the rows
        row_dict = dict()

        # look for the item metadata, assign it to the dictionary; 
        # start with some basic elements likely (already enumerated in the headers list) :
        # item type
        row_dict['item_type'] = item_type
        # date uploaded
        row_dict['date_uploaded'] = date_uploaded
        # source filename
        row_dict['source_file'] = source_file
        # identifier
        row_dict['item_id'] = item_id
        # title
        row_dict['title'] = title
        # date
        row_dict['date'] = date
        # link
        row_dict['source_url'] = source_url
        # format
        row_dict['phys_format'] = phys_format
        # digital format
        row_dict['dig_format'] = dig_format.capitalize()
        # rights
        row_dict['rights'] = rights
        # image
        row_dict['image_url'] = image_url
        # creators
        row_dict['creators'] = creators
        # description
        row_dict['description'] = description
        # subject headings
        row_dict['subject_headings'] = subject_headings
        # source collection
        row_dict['source_collection'] = source_collection
        # url
        row_dict['url'] = url
        # notes
        row_dict['notes'] = notes

        # write to the csv
        with open(collection_info_csv, 'a', encoding='utf-8') as fout:
            writer = csv.DictWriter(fout, fieldnames=headers, lineterminator='\n')
            if items_written == 0:
                writer.writeheader()
            writer.writerow(row_dict)
            items_written += 1
            print('adding',item_id)

print('\n\n--- LOG ---')
print('wrote',collection_info_csv)
print('with',items_written,'items')
print(error_count,'errors (info not written)')

removed collection_items_data_gardens.csv
opening item-metadata-gardens\item_metadata-afc1999008.afc1999008_crf_mhc05407.json
adding cmns001121
opening item-metadata-gardens\item_metadata-cph.3a25266.json
adding 2001703638
opening item-metadata-gardens\item_metadata-cph.3c30923.json
adding 2017865756
opening item-metadata-gardens\item_metadata-cph.3f05737.json
adding 99400959
opening item-metadata-gardens\item_metadata-cph.3g05158.json
adding 97502889
opening item-metadata-gardens\item_metadata-ds.03659.json
adding 2013645840
opening item-metadata-gardens\item_metadata-ds.13761.json
adding 2020630497
opening item-metadata-gardens\item_metadata-fsa.8a19113.json
adding 2017732863
opening item-metadata-gardens\item_metadata-fsa.8b14071.json
adding 2017762632
opening item-metadata-gardens\item_metadata-fsa.8b14514.json
adding 2017764422
opening item-metadata-gardens\item_metadata-fsa.8c19781.json
adding 2017812834
opening item-metadata-gardens\item_metadata-fsa.8d21091.json
adding 20178321

Now, you should have a well-formed, complete CSV file at `data/collection_items_data.csv`. This file should have all the information to import the 59 items that you were able to identify. 